<a href="https://colab.research.google.com/github/stevegbrooks/commodify/blob/preprocessing/usda_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests, zipfile, io
import pandas as pd
out_path = "~/CIS550/commodify/data/"

## Process commodities data from USDA

In [2]:
zip_url = "https://apps.fas.usda.gov/psdonline/downloads/psd_alldata_csv.zip"

r = requests.get(zip_url)
if r.ok:
  z = zipfile.ZipFile(io.BytesIO(r.content))
  usda_data = pd.read_csv(z.open('psd_alldata.csv'))

usda_data.head(n=5)

,Commodity_Code,Commodity_Description,Country_Code,Country_Name,Market_Year,Calendar_Year,Month,Attribute_ID,Attribute_Description,Unit_ID,Unit_Description,Value
0,577400,"Almonds, Shelled Basis",AF,Afghanistan,2010,2018,10,20,Beginning Stocks,21,(MT),0.0
1,577400,"Almonds, Shelled Basis",AF,Afghanistan,2010,2018,10,125,Domestic Consumption,21,(MT),0.0
2,577400,"Almonds, Shelled Basis",AF,Afghanistan,2010,2018,10,176,Ending Stocks,21,(MT),0.0
3,577400,"Almonds, Shelled Basis",AF,Afghanistan,2010,2018,10,88,Exports,21,(MT),0.0
4,577400,"Almonds, Shelled Basis",AF,Afghanistan,2010,2018,10,57,Imports,21,(MT),0.0


### Deal with null values

In [3]:
usda_data.isnull().values.any()

True

It looks like only the `Country_Code` column has `NaN`. 


In [4]:
usda_data[usda_data.isnull().any(axis=1)]

,Commodity_Code,Commodity_Description,Country_Code,Country_Name,Market_Year,Calendar_Year,Month,Attribute_ID,Attribute_Description,Unit_ID,Unit_Description,Value
716727,813100,"Meal, Soybean",NaN,Netherlands Antilles,1976,2006,6,20,Beginning Stocks,8,(1000 MT),0.0
716728,813100,"Meal, Soybean",NaN,Netherlands Antilles,1976,2006,6,7,Crush,8,(1000 MT),0.0
716729,813100,"Meal, Soybean",NaN,Netherlands Antilles,1976,2006,6,125,Domestic Consumption,8,(1000 MT),1.0
716730,813100,"Meal, Soybean",NaN,Netherlands Antilles,1976,2006,6,176,Ending Stocks,8,(1000 MT),0.0
716731,813100,"Meal, Soybean",NaN,Netherlands Antilles,1976,2006,6,88,Exports,8,(1000 MT),0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1726314,612000,"Sugar, Centrifugal",NaN,Netherlands Antilles,2021,2020,11,99,Refined Exp.(Raw Val),8,(1000 MT),0.0
1726315,612000,"Sugar, Centrifugal",NaN,Netherlands Antilles,2021,2020,11,74,Refined Imp.(Raw Val),8,(1000 MT),0.0
1726316,612000,"Sugar, Centrifugal",NaN,Netherlands Antilles,2021,2020,11,126,Total Disappearance,8,(1000 MT),0.0
1726317,612000,"Sugar, Centrifugal",NaN,Netherlands Antilles,2021,2020,11,178,Total Distribution,8,(1000 MT),0.0


Lets check which values for `Country_Name` don't have a `Country_Code`

In [5]:
usda_data[usda_data.isnull().any(axis=1)]["Country_Name"].unique()

array(['Netherlands Antilles'], dtype=object)

Before gaining indepedence in 2010, these islands were part of the Netherlands, but now the group of islands consists of smaller countries. 

We can just set the country code of Netherlands Antilles to the Netherlands' country code.

In [6]:
usda_data[usda_data["Country_Name"] == "Netherlands"]["Country_Code"].unique()

array(['NL'], dtype=object)

In [7]:
usda_data.loc[usda_data.Country_Name == "Netherlands Antilles", 'Country_Code'] = "NL"

Check to make sure it worked.

In [8]:
usda_data.isnull().values.any()

False

## Reshape from long to wide

In [37]:
usda_pivot = usda_data.pivot(index = ["Commodity_Description", "Market_Year", "Month", "Country_Name"], columns = "Attribute_Description", values = "Value")


In [43]:
usda_pivot_reset = usda_pivot.reset_index(drop=True)
usda_pivot_reset

Attribute_Description,Annual % Change Per Cap. Cons.,Annual % Change Prod. To Sows,Arabica Production,Area Harvested,Bean Exports,Bean Imports,Beef Cows Beg. Stocks,Beet Sugar Production,Beginning Stocks,Calf Slaughter,...,TY Imp. from U.S.,TY Imports,Total Disappearance,Total Distribution,Total Slaughter,Total Supply,Total Use,USE Dom. Consumption,Withdrawal From Market,Yield
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,0.0,NaN,4.0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,0.0,NaN,13.0,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,0.0,NaN,2.0,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,0.0,NaN,1.0,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,0.0,NaN,29.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146458,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,142.0,NaN,...,0.0,1000.0,NaN,1142.0,NaN,1142.0,NaN,NaN,NaN,0.00
146459,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,799.0,NaN,...,0.0,3600.0,NaN,4399.0,NaN,4399.0,NaN,NaN,NaN,0.00
146460,NaN,NaN,NaN,95.0,NaN,NaN,NaN,NaN,564.0,NaN,...,0.0,3800.0,NaN,4502.0,NaN,4502.0,NaN,NaN,NaN,1.45
146461,NaN,NaN,NaN,30.0,NaN,NaN,NaN,NaN,18.0,NaN,...,0.0,50.0,NaN,258.0,NaN,258.0,NaN,NaN,NaN,6.33


In [44]:
cols_to_keep = ["Commodity_Description", "Market_Year", "Month", "Country_Name",
           "Beginning Stocks", "Ending Stocks", "Imports", "Exports", 
           "Area Harvested", "Yield", "Production", "Domestic Consumption"]
usda_pivot_reset[cols_to_keep].to_csv(out_path + "usda_data.csv")

KeyError: "['Country_Name', 'Market_Year', 'Month', 'Commodity_Description'] not in index"